# Generalized Linear Models

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import pandas as pd
from sklearn.model_selection import train_test_split

## Beyond Linear Regression

Linear regressions have [limitations](https://en.wikipedia.org/wiki/Generalized_linear_model).

As it stands, the algorithm could generate a prediction *anywhere on the real number line*. This *may* be realistic, like if I'm predicting national surpluses/debts.

But what if I'm predicting values of a variable that doesn't take, say, negative values, like temperature in Kelvin?

What if I'm predicting values of a variable that takes only integer values, like the number of mouseclicks on my killer ds blog per minute?

What if I'm predicting probabilities? Or something Boolean / Bernoullian?

What if the shape of my errors changes as a function of the dependent variable?

Am I stuck using linear regression? There's got to be a better way!

The strategy now is to *generalize* the notion of linear regression; linear regression as we've known it will become a special case. In particular, we'll keep the idea of the regression best-fit line, but now **we'll allow the model to make predictions through some (non-trivial) transformation of the linear predictor**.

Let's say we've constructed our best-fit line, i.e. our linear predictor, $\hat{L} = \beta_0 + \beta_1x_1 + ... + \beta_nx_n$.

## Logistic Regression

Consider the following transformation: <br/>
$\large\hat{y} = \Large\frac{1}{1 + e^{-\hat{L}}} \large= \Large\frac{1}{1 + e^{-(\beta_0 + ... + \beta_nx_n)}}$. This is called the **sigmoid function**.

We're imagining that $\hat{L}$ can take any values between $-\infty$ and $\infty$.

$\large\rightarrow$ But what values can $\hat{y}$ take? What does this function even look like?

In [15]:
# Let's plot this function here:

X = np.linspace(-10, 10, 300)
Y = 1 / (1 + np.exp(-X))

plt.figure(figsize=(8, 6))
plt.plot(X, Y, 'r');

### Interpretation

This function squeezes our predictions between 0 and 1. And that's why it's so useful for **binary classification problems**.

Suppose I'm building a model to predict whether a plant is poisonous or not, based perhaps on certain biological features of its leaves. I'll let '1' indicate a poisonous plant and '0' indicate a non-poisonous plant.

Now I'm forcing my predictions to be between 0 and 1, so suppose for test plant $P$ I get some value like 0.19.

I can naturally understand this as **the probability that $P$ is poisonous**.

If I truly want a binary prediction, I can simply round my score appropriately.

How do we fit a line to our dependent variable if its values are already stored as probabilities? We can use the inverse of the sigmoid function, and just set our regression equation equal to that. The inverse of the sigmoid function is called the **logit function**, and it looks like this:

$\large f(y) = \ln\left(\frac{y}{1 - y}\right)$. Notice that the domain of this function is $(0, 1)$.

$\hspace{110mm}$(Quick proof that logit and sigmoid are inverse functions:

$\hspace{170mm}x = \frac{1}{1 + e^{-y}}$; <br/>
$\hspace{170mm}$so $1 + e^{-y} = \frac{1}{x}$; <br/>
$\hspace{170mm}$so $e^{-y} = \frac{1 - x}{x}$; <br/>
$\hspace{170mm}$so $-y = \ln\left(\frac{1 - x}{x}\right)$; <br/>
$\hspace{170mm}$so $y = \ln\left(\frac{x}{1 - x}\right)$.)

Our regression equation will now look like this:

$\large\ln\left(\frac{y}{1 - y}\right) = \beta_0 + \beta_1x_1 + ... + \beta_nx_n$.

This equation is used for a **logistic regression**. Note that it is now not the target variable itself that is modeled as varying linearly with the predictor(s) but rather the values of this logit function of the target that are so represented. This is the sense in which we have a more generalized notion of a linear model.

This function whose values are modeled as varying linearly is in general called the **link function**. Logistic regression's link function is the logit function, but different sorts of models use different link functions. We'll look at another example below.

[Wikipedia](https://en.wikipedia.org/wiki/Generalized_linear_model#Link_function) has a nice table of generalized linear model types and their associated link functions.

### Odds

There are other ways to squeeze the results of a linear regression into the set (0, 1).

But the ratio $\frac{p}{1-p}$ represents the *odds* of some event, where $p$ is the probability of the event. And so *this* logit function represents the **log-odds** of success (y=1).

### Cost Functions and Solutions to the Optimization Problem

No one has yet found a closed-form solution to the optimization problem presented by logistic regression. But even if one exists, the computation would no doubt be so complex that we'd be better off using some sort of approximation method instead.

Various versions of gradient descent or coordinate descent (this is like gradient descent but it focuses only on one parameter at a time) have been used. The scikit-learn class expects the user to specify the solver to be used in calculating the coefficients.

Question: What are we using this approximation method *on*? With linear regression we could sensibly calculate a residual sum of squares, but that doesn't seem to apply any more.

Roughly, we want to measure how far off our predictions are. (That part is still the same.) But now we'll be comparing our predictions to 0's and 1's. Predictions near 0 for actual negatives and near 1 for actual positives should count far less to our loss function than predictions near 1 for actual negatives and near 0 for actual positives. See [here](https://towardsdatascience.com/optimization-loss-function-under-the-hood-part-ii-d20a239cde11) for more details.

### Logistic Regression in Sci-Kit Learn

In [17]:
from sklearn.linear_model import LogisticRegression

data = pd.read_csv('heart.csv')

X = data.drop('target', axis=1)
y = data['target']

data.head()

In [5]:
# Let's split our data into train and test.

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [18]:
# Instantiate a logistic regression object with the 'liblinear' solver,
# which is good for small datasets.

logreg = LogisticRegression(solver='liblinear', multi_class='auto')

# Now fit it to the training data.

logreg.fit(X_train, y_train)

#### Coefficients

In [19]:
logreg.coef_

In [20]:
logreg.intercept_

#### `.predict()` and `.predict_proba_()`

In [21]:
first_test_row = X_test.head(1)
logreg.predict(first_test_row)

In [22]:
logreg.predict_proba(first_test_row)

We should be able to reproduce the prediction if we plug our betas into the sigmoid function!

In [23]:
line_value = logreg.coef_.dot(X_test.head(1).T) + logreg.intercept_

1 / (1 + np.exp(-line_value))

## Poisson Regression

Let's get a taste of a different sort of generalized linear model. Here's a new regression equation:

$\large\ln(y) = \beta_0 + \beta_1x_1 + ... + \beta_nx_n$. The link function is simply $\ln(y)$ and so we have:

$\large\hat{y} = e^\hat{L} = e^{\beta_0 + ... + \beta_nx_n}$.

The domain, or "support", for a Poisson distribution is {0, 1, 2, ... }. Can you see why?

### Poisson Regression in Statsmodels

In [30]:
awards = pd.read_csv('https://stats.idre.ucla.edu/stat/data/poisson_sim.csv')

In [24]:
awards.head()

What is this dataset about?

The data show the number of awards earned by students at one high school. 'Prog' is a coded version of the sort of program in which the student was enrolled and 'math' is a score on a math exam.

Let's one-hot encode it:

In [15]:
from sklearn.preprocessing import OneHotEncoder

In [24]:
ohe = OneHotEncoder(categories='auto')

ohe_new = ohe.fit_transform(awards['prog'].values.reshape(-1, 1))

In [32]:
awards_dums = pd.concat([awards, pd.DataFrame(ohe_new.todense())], axis=1)

In [25]:
awards_dums.head()

In [26]:
# Get a statsmodels summary here!

X = sm.add_constant(awards_dums[['math', 0, 1, 2]])
y = awards_dums['num_awards']

poi_model = sm.GLM(y, X, sm.families.Poisson())
poi_model.fit().summary()

In [27]:
# Interpreting the results

np.exp(0.0702)